In [ ]:
import cv2
import numpy as np

# Function to track objects using markers (e.g., QR code or April tags)
def track_with_marker(video_source):
    # Initialize video capture
    cap = cv2.VideoCapture(video_source)

    # Load marker detection dictionary
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)

    # Load calibration parameters (if available)
    camera_matrix = np.loadtxt('camera_matrix.txt')
    distortion_vector = np.loadtxt('distortion_vector.txt')

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect markers
        corners, ids, _ = cv2.aruco.detectMarkers(gray, aruco_dict)

        if ids is not None:
            # Draw markers
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Estimate pose of markers
            rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 0.05, camera_matrix, distortion_vector)

            # Draw axis for each marker
            for i in range(len(rvecs)):
                cv2.aruco.drawAxis(frame, camera_matrix, distortion_vector, rvecs[i], tvecs[i], 0.1)

        cv2.imshow("Marker Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Function to track objects without markers
def track_without_marker(video_source):
    # Initialize video capture
    cap = cv2.VideoCapture(video_source)

    # Load pre-trained object detection model (e.g., YOLO)
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = f.read().splitlines()

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        height, width, _ = frame.shape

        # Preprocess frame for object detection
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        net.setInput(blob)

        # Perform object detection
        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)

        # Process detections
        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, f'{classes[class_id]} {int(confidence * 100)}%', (x, y - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("Object Tracking", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to choose between tracking with or without marker
if __name__ == "__main__":
    # Choose video source (e.g., webcam or video file)
    video_source = 0  # Use webcam
    #video_source = "path_to_video_file.mp4"

    # Choose tracking method
    method = input("Choose tracking method (1: with marker, 2: without marker): ")

    if method == "1":
        track_with_marker(video_source)
    elif method == "2":
        track_without_marker(video_source)
    else:
        print("Invalid choice. Choose either 1 or 2.")